# Orion - List

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df_list = tabula.read_pdf("2018_offenlegung-geldwerter-leistungen_final-version.pdf", pages='all', lattice=True, multiple_tables=True)
df = pd.concat(df_list)

## Format Table

In [3]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['name', 'location', 'country', 'address', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'total', 'empty1']

#Shift all where 'name' is empty
df_export[df_export['name'].isna()] = df_export[df_export['name'].isna()].shift(-1, axis='columns')

#Remove empty rows
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.dropna(subset=['name'], how='all')

#Remove empties
df_export.drop(columns=['empty1'], inplace=True)

#Add type
df_export = set_type_by_alphabetical_order(df_export)

#Remove PLZ from address
for index, row in df_export.iterrows():
    adr_split = row['address'].split(',')
    if len(adr_split) > 1:
        plz_ort = adr_split[len(adr_split) - 1].strip()
        df_export.loc[index, 'plz'] = plz_ort.split(' ')[0]
        addr = row['address'].replace(',' + plz_ort, '')
        addr = addr.replace(', ' + plz_ort, '')
        df_export.loc[index, 'address'] = addr

#Cleanup Numbers and sum
df_export = remove_in_numbers(df_export, '-')
df_export = remove_in_numbers(df_export, 'N/A')
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

#Add empty fields
df_export = add_uci(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Revert name
df_export = revert_name(df_export, ' ')

#Remove Comma
df_export['name'] = df_export['name'].str.replace(",", "")
df_export['name'] = df_export['name'].str.strip()

export_list(df_export, 'orion')

Total nicht Summe der Werte
saved


In [6]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx', open=True)